# Getting the journey data

The TfL bike usage data is hosted as a number of CSV files on their website. I looped through each of these CSVs, aggregated them together into one dataset, and cleaned it all up.

In [ ]:
import pandas as pd
import urllib

## Read list of names of all files from a separate CSV - can't scrape them from website as they're hidden
with open('urls_bike.csv', 'r') as f:
    csv_list = f.read().splitlines()

## Downloading all of the bike journey CSV files and appending to one dataset
website = 'http://cycling.data.tfl.gov.uk/usage-stats/'

url_list = [website + urllib.parse.quote(x) for x in csv_list]
dfs = (pd.read_csv(url) for url in url_list)
all_data = pd.concat(dfs, ignore_index=True)

print(all_data.shape)
all_data.head()

In [ ]:
## Clean data - NAs, and start/end being the same station, drop additional useless columns

print(all_data.shape)

all_data.dropna(axis=0, subset=["StartStation Id", "EndStation Id", "Start Date", "End Date"], inplace=True)

print(all_data.shape)

all_data["EndStation Id"] = all_data["EndStation Id"].astype(int)
all_data["StartStation Id"] = all_data["StartStation Id"].astype(int)

all_data = all_data[all_data["StartStation Id"] != all_data["EndStation Id"]]

all_data = all_data.loc[:,('Start Date',
                           'StartStation Id',
                           'End Date',
                           'EndStation Id',
                           'Duration')]
                           
print(all_data.shape)

## Extra drop for duplicates

all_data.drop_duplicates(inplace=True)
print(all_data.shape)

all_data.head()

# Getting the bike station locations

TfL have a live "cycle hire updates" feed which lists information for each cycle hire station, updated once every minute or so. I don't utilise this live data - instead I just take the name, ID, lat/lon, and capacity for each bike station.

In [ ]:
import requests
from xml.etree import ElementTree as ET
import pandas as pd

site = "https://tfl.gov.uk/tfl/syndication/feeds/cycle-hire/livecyclehireupdates.xml"

response = requests.get(site)
root = ET.fromstring(response.content)

id_list = [int(root[i][0].text) for i in range(0, len(root))]
name_list = [root[i][1].text for i in range(0, len(root))]
lat_list = [float(root[i][3].text) for i in range(0, len(root))]
lon_list = [float(root[i][4].text) for i in range(0, len(root))]
capacity_list = [int(root[i][12].text) for i in range(0, len(root))]

all_locs = pd.DataFrame(list(zip(name_list, id_list, lat_list, 
                                 lon_list, capacity_list)), columns = ["name","id","lat","lon","capacity"])

all_locs.to_csv('bike_point_locations_saved.csv', header=True, index=None)

print(all_locs.shape)
all_locs.head()

## Plotting all the bike stations in bokeh

Once I've got all the bike station locations, I generate a quick interactive bokeh plot of all of them. For the backgrounds I use two separate shapefiles I downloaded - one of all the buildings in London, and one of all the roads. I've got a separate github repo which goes through how I configured all of these.

In [ ]:
from bokeh.models import GeoJSONDataSource, ColumnDataSource
from bokeh.models.tools import PanTool, HoverTool, ResetTool, WheelZoomTool
from bokeh.io import output_notebook, output_file, save, show
from bokeh.sampledata.sample_geojson import geojson
import bokeh.plotting as bp
import json

In [ ]:
## Load both roads and buildings geojson files into correct format 

# Load buildings but truncate heavily to 100 polygons (at first)
with open('Basemaps/London_buildings.geojson', 'r') as f:
    geojson_buildings = f.read()
    
with open('Basemaps/London_roads.geojson', 'r') as f:
    geojson_roads = f.read()

# Load geojson
json_buildings = GeoJSONDataSource(geojson=json.dumps(json.loads(geojson_buildings)))
json_roads = GeoJSONDataSource(geojson=json.dumps(json.loads(geojson_roads)))

In [ ]:
# Sort out the dataframe for plotting - bounding box, size of plot, etc

all_locs = pd.read_csv('bike_point_locations_saved.csv')

df_points = all_locs.loc[:,('name', 'capacity','lat', 'lon')]
df_points['size'] = 5

x_range = (df_points.lon.min() - 0.001, df_points.lon.max() + 0.003)
y_range = (df_points.lat.min() - 0.003, df_points.lat.max() + 0.003)

points_source = ColumnDataSource(ColumnDataSource.from_df(df_points))

print(x_range)
print(y_range)

plot_h = 600
plot_w = 900

df_points.head()

In [ ]:
## Set up the bokeh plot. I use the "hovertool" feature from bokeh so that I can mouseover the points and get their
## name and capacity. I had to play around with the circle renderer to get the hovertool to work properly, as 
## otherwise it interacts with the other elements in my bokeh plot and returns null values - i.e. if a bike point
## sits on top of one of the road of building patches, then the hovertool was returning a null value. 

## This function seems to yield a bunch of GlyphRenderer errors, but the plot works exactly as intended...

tools = [PanTool(), WheelZoomTool(), ResetTool()]

p = bp.figure(tools=tools, plot_width=plot_w, plot_height=plot_h,
    x_range=x_range, y_range=y_range, outline_line_color=None,
    min_border=0, min_border_left=0, min_border_right=0,
    min_border_top=0, min_border_bottom=0) 

p.patches(xs='xs', ys='ys', fill_alpha=0.3, fill_color='#0C090A',
                   line_alpha=0, source=json_buildings)

circles = p.circle(x='lon', y='lat', size='size', color='#ff0000', alpha=1, source=points_source)

hover = HoverTool(
        tooltips=[
            ("Name", "@name"),
            ("Capacity", "@capacity")
        ]
    )

p.add_tools(hover)


p.background_fill_color = '#2C3539'
p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
hover.renderers.append(circles)

output_file("bokeh_plots/bike_points.html")
save(p)

output_notebook()
show(p)

## Combining the usage data and location data

I calculate the full set of unique routes actually made within the entire usage data, so that I can then run these unique routes through my journey planner. I get around 400k unique routes made - out of a total possible of around 600k (777 * 776).

In [ ]:
## Generate list of unique routes
unq_locs = all_data.loc[:,('StartStation Id',
                      'EndStation Id')]
print(unq_locs.shape)
unq_locs.drop_duplicates(inplace=True)
print(unq_locs.shape)

## Merge on the lat/lons

unq_locs = unq_locs.merge(right = all_locs,
                             how = 'inner',
                             left_on = 'StartStation Id',
                             right_on = 'id')

print(unq_locs.shape)

unq_locs.drop(labels = ["id", "name"], axis=1, inplace=True)
unq_locs.rename(columns={'lat': 'StartStation lat', 'lon': 'StartStation lon', 
                            'capacity': 'StartStation capacity'},
                   inplace=True)
# Merge end
unq_locs = unq_locs.merge(right = all_locs,
                             how = 'inner',
                             left_on = 'EndStation Id',
                             right_on = 'id')

unq_locs.drop(labels = ["id", "name"], axis=1, inplace=True)
unq_locs.rename(columns={'lat': 'EndStation lat', 'lon': 'EndStation lon',
                           'capacity': 'EndStation capacity'},
                   inplace=True)


print(unq_locs.shape)
unq_locs.head()

# Calculating the predicted route for each journey taken

I run each unique route through an OSRM server to calculate the predicted route taken, as well as its distance and expected duration. The result is a list of lat/lon waypoints for each route. 

DATASHADER NOTE - datashader's "line" drawing function takes a series of lat/lon points and draws a line between each one contiguously. Unfortunately, this means that it draws a line between the end of one journey and the start of the next, like if you were trying to draw a picture without taking your pen off the paper. To get around this, I append a null row at the end of every unique route.

In [ ]:
## Define functions to do routing requests to OSRM server - use asynchronous requests to speed it up
import aiohttp
import asyncio
import numpy as np
concurrent = 500

## Creating the URL for each individual route to send to the OSRM server
def CreateUrl(row):
    sid, slat, slon, eid, elat, elon = row
    base_url = "http://osmrouting.uksouth.cloudapp.azure.com/"
    route_url = "osrm/route/v1/bicycle/{0},{1};{2},{3}".format(slon, slat, elon, elat)
    end_url = "?alternatives=false&steps=false&geometries=geojson"
    return base_url+route_url+end_url, '%i_%i' % (sid, eid)

url_list = unq_locs.apply(func=CreateUrl, axis=1)
print(len(url_list))

df_empty = pd.DataFrame(index=np.arange(0, 1), columns=('lat', 'lon', 'duration', 'distance'))

## Handling the JSON request - need to insert an empty row in between each route 
def handle_req(data, qid):
    r = json.loads(data.decode('utf-8'))
    coord = r['routes'][0]['geometry']['coordinates']
    distance = r['routes'][0]['distance']
    duration = r['routes'][0]['duration']
        
    df_journey = pd.DataFrame(coord[:], columns=['lon', 'lat'])
    df_journey['duration'] = duration
    df_journey['distance'] = distance
    df_journey = df_journey.append(df_empty)
    
    df_journey['route_id'] = qid

    return df_journey
 
## Asynchronous request functions
def chunked_http_client(num_chunks, s):
    # Use semaphore to limit number of requests
    semaphore = asyncio.Semaphore(num_chunks)
    @asyncio.coroutine
    # Return co-routine that will work asynchronously and respect locking of semaphore
    def http_get(url, qid):
        nonlocal semaphore
        with (yield from semaphore):
            response = yield from s.get(url)
            body = yield from response.content.read()
            yield from response.wait_for_close()
        return body, qid
    return http_get
 
def run_experiment(urls, _session):
    http_client = chunked_http_client(num_chunks=concurrent, s=_session)
    
    # http_client returns futures, save all the futures to a list
    tasks = [http_client(url, qid) for url, qid in urls]
    
    df_route = []
    
    # wait for futures to be ready then iterate over them
    for future in asyncio.as_completed(tasks):
        data, qid = yield from future
        try:
            out = handle_req(data, qid)
            df_route.append(out)
        except Exception as err:
            print("Error for {0} - {1}".format(qid, err))
    return df_route

In [ ]:
## Send all of the requests to the OSRM server and compile the results together

with aiohttp.ClientSession() as session: 
    loop = asyncio.get_event_loop()
    calc_routes = loop.run_until_complete(run_experiment(url_list, session))
    
routes_complete = pd.concat(calc_routes, ignore_index=True)

routes_complete.to_csv('all_id_routes.csv', header=True, index=None)

print(routes_complete.shape)
routes_complete.head()

## Filtering and collapsing the dataset

Using the results from my predicted journeys, for the datashader plot I filter the dataset by only taking routes that took up to twice as long as predicted by the route planner (i.e. filtering out routes where the rider clearly went in an entirely different direction, or some very slow people).

I also define a bunch of other functions below for filtering the dataset on e.g. time of day, removing weekends, collapsing by unique route.

In [ ]:
## Reading the output of the above cells from the saved CSV file, then generating a separate dataset just of the 
## expected duration for each route

routes_complete = pd.read_csv('all_id_routes.csv')

routes_duration = routes_complete.loc[:,("route_id","duration")]
print(routes_duration.shape)

routes_duration.drop_duplicates(inplace=True)
print(routes_duration.shape)

## Dropping the NaN values
routes_duration.dropna(axis=0,inplace=True)
print(routes_duration.shape)

routes_duration.head()

In [ ]:
## Add on a route_id variable and use it to merge on the expected duration

all_data['route_id'] = all_data['StartStation Id'].astype(str) + "_" + all_data['EndStation Id'].astype(str)

print(all_data.shape)
all_data = all_data.merge(right = routes_duration, how='inner', on='route_id')
all_data.rename(columns={'duration': 'Expected Duration'},
                   inplace=True)

## Set upper limit at double the expected time for each unique route
all_data['duration_upper'] = (2*all_data['Expected Duration'])

print(all_data.shape)

## Used for collapsing stuff later
all_data['c'] = 1

In [ ]:
## Add some extra variables to the dataset for use later in filtering

import datetime

## Feeding a specififed date format speeds up the pd.to_datetime function immeasurably, especially over large datasets
## e.g. http://stackoverflow.com/questions/32034689/why-is-pandas-to-datetime-slow-for-non-standard-time-format-such-as-2014-12-31

format = "%d/%m/%Y %H:%M"

## Some routes had dates with a seconds component, whereas some didn't - the below code cuts these seconds off
all_data['Start Date']= all_data['Start Date'].str[:16]

all_data['Start Date Converted']= pd.to_datetime(all_data['Start Date'], format=format).dt.date

all_data['Hours']= pd.to_datetime(all_data['Start Date'], format=format).dt.hour

all_data['Day']= pd.to_datetime(all_data['Start Date'], format=format).dt.weekday

all_data.head()

In [ ]:
## All_routes is the subset where I cut out those over the expected threshold

all_routes = all_data[all_data['Duration'] < all_data['duration_upper']].copy()

del all_routes['duration_upper']
del all_data['duration_upper']


In [ ]:
## This allows me to filter the dataset to between a specific time of day
def time_filter(dataset, lower_time, upper_time):
    dataset = dataset[(dataset['Hours'] >= lower_time) & (dataset['Hours'] < upper_time)]
    return dataset

## This does the same but for certain dates
def date_filter(dataset, lower_year, lower_month, lower_day, upper_year, upper_month, upper_day):
    dataset = dataset[(dataset["Start Date Converted"] >= datetime.datetime(lower_year,lower_month,lower_day).date()) 
                      & (dataset["Start Date Converted"] < datetime.datetime(upper_year,upper_month,upper_day).date())]
    return dataset

## This cuts out weekends 
def weekend_filter(dataset):
    dataset = dataset[(dataset["Day"] < 5)]
    return dataset

## This drops one specific day
def drop_specific_date(dataset, year, month, day):
    dataset = dataset[(dataset["Start Date Converted"] != datetime.datetime(year,month,day).date())]
    return dataset

## This collapses down any cut of a dataset to get the count by route for each route_id
def collapse_dataset(dataset, renamed_count):
    dataset_collapse = dataset.loc[:, ('route_id', 'c')].groupby(['route_id']).sum()
    dataset_collapse.reset_index(inplace=True)
    dataset_collapse.rename(columns={'c': str(renamed_count)}, inplace=True)
    return dataset_collapse

## This collapses the dataset down and just looks and inflows and outflows for each station
def inflow_outflow(dataset):
    inflows = dataset.loc[:, ('EndStation Id', 'c')].copy().groupby(['EndStation Id']).sum()
    inflows.reset_index(inplace=True)
    inflows.rename(columns={'c': 'inflows', 'EndStation Id': 'id'}, inplace=True)

    outflows = dataset.loc[:, ('StartStation Id', 'c')].copy().groupby(['StartStation Id']).sum()
    outflows.reset_index(inplace=True)
    outflows.rename(columns={'c': 'outflows', 'StartStation Id': 'id'}, inplace=True)
    flows = inflows.merge(right = outflows, how='inner', on='id')
    
    flows['delta'] = flows['inflows'] - flows['outflows']

    return flows 

## Plotting the entire dataset

Using the functions above, I collapse the entire dataset down, merge it onto the waypoints for each route, then make a simple plot using datashader

In [ ]:
## Collapse down entire dataset, then merge onto the waypoints for each route

print(all_routes.shape)

collapsed_routes = collapse_dataset(all_routes, 'c_all')

print(collapsed_routes.shape)

agg_routes = collapsed_routes.merge(right = routes_complete, how='inner', on='route_id')

print(agg_routes.shape)

In [ ]:
## Plot the resulting dataset using datashader, aggregating the lines by each route's count

import datashader as ds
import datashader.transfer_functions as tf
from datashader.bokeh_ext import InteractiveImage

df = agg_routes.loc[:,('lat', 'lon', 'c_all')]

# Default plot ranges:

x_range = (df.lon.min() - 0.001, df.lon.max() + 0.001)
y_range = (df.lat.min() - 0.0005, df.lat.max() + 0.0005)

plot_w = 900
plot_h = 600

background_tools = [PanTool(), WheelZoomTool(), ResetTool()]

background = bp.figure(tools=background_tools, plot_width=plot_w, plot_height=plot_h,
    x_range=x_range, y_range=y_range, outline_line_color=None,
    min_border=0, min_border_left=0, min_border_right=0,
    min_border_top=0, min_border_bottom=0) 

background.patches(xs='xs', ys='ys', fill_alpha=0.3, fill_color='#0C090A',
                   line_alpha=0, source=json_buildings)

background.background_fill_color = '#2C3539'
background.xaxis.visible = False
background.yaxis.visible = False
background.xgrid.grid_line_color = None
background.ygrid.grid_line_color = None


def create_image(x_range=x_range, y_range=y_range, w=plot_w, h=plot_h):
    cvs = ds.Canvas(x_range=x_range, y_range=y_range, plot_height=h, plot_width=w)
    agg = cvs.line(df, 'lon', 'lat', agg=ds.count('c_all'))
    return tf.shade(agg, cmap=["darkred", "red"])

InteractiveImage(background, create_image)

# ANALYSIS

Now that I have a completed dataset, I can make various cuts to the dataset to see commuting patterns and the effects of tube strikes.

In [ ]:
## Dataset of just the names of stations

all_data_names = all_locs.loc[:, (['id', 'name'])].copy()
all_data_names.rename(columns={'name': 'Name'}, inplace=True)
all_data_names.head()

In [ ]:
## Look at most bikes taken across entire dataset

all_flows = inflow_outflow(all_data)

all_flows['Total Flows'] = all_flows['inflows'] + all_flows['outflows']

all_flows.sort_values('Total Flows', axis=0, inplace=True, ascending=False)

all_flows = all_flows.merge(right = all_data_names, how='left', on='id')

all_flows = all_flows[['Name', 'Total Flows']][:10]

all_flows[:10]

all_flows.head()


In [ ]:
## Can also use the PageRank algorithm 
import networkx as nx

data_for_rank = all_data.loc[:, ('StartStation Id', 'EndStation Id', 'c')].groupby(['StartStation Id', 'EndStation Id']).sum()
data_for_rank.reset_index(inplace=True)
data_for_rank.rename(columns={'c': 'Count'}, inplace=True)

data_for_rank.head()

network = nx.from_pandas_dataframe(data_for_rank, 'StartStation Id', 'EndStation Id', edge_attr=['Count'])

pagerank = pd.DataFrame(list(nx.pagerank(network).items()), columns = ['id', 'Rank'])

pagerank = pagerank.merge(right=all_data_names, how='left', on='id')

pagerank.sort_values('Rank', axis=0, inplace=True, ascending=False)
pagerank.reset_index(inplace=True, drop=True)

pagerank = pagerank.loc[:, ('Name', 'Rank')]

pagerank[:10]

In [ ]:
## Looking at stations with highest redistribution

redistribution = inflow_outflow(all_data)
redistribution.sort_values('delta', axis=0, inplace=True)
redistribution.reset_index(inplace=True, drop=True)

redistribution = redistribution.merge(right=all_data_names, how='left', on='id')
redistribution = redistribution.loc[:, (['Name', 'delta'])]
redistribution.rename(columns={'delta': 'Inflows - Outflows'}, inplace=True)

redistribution[:10]

## Look at mornings vs evenings across the dataset to see different flows

Would expect to see people commuting into the centre in the morning, and back out to the suburbs in the evenings. Cut the dataset by morning/evening, filter out weekends, then produce an interactive plot of morning vs evening

In [ ]:
## Filter dataset to only look at mornings and evenings to see commuting patterns

all_mornings = weekend_filter(all_data)
all_mornings = time_filter(all_mornings, 7, 10)

all_evenings = weekend_filter(all_data)
all_evenings = time_filter(all_evenings, 17, 20)

all_mornings_flows = inflow_outflow(all_mornings)
all_evenings_flows = inflow_outflow(all_evenings)

all_mornings_flows.rename(columns={'delta': 'mornings'}, inplace=True)
all_evenings_flows.rename(columns={'delta': 'evenings'}, inplace=True)

mornings_evenings = all_mornings_flows.merge(right=all_evenings_flows, on='id', how='inner')

## Below function sets colour as red if more inflows than outflows, and green if outflows > inflows
mornings_evenings['c_mornings'] = mornings_evenings.apply(lambda x: 'red' if x['mornings'] < 0 else 'green', axis=1)
mornings_evenings['c_evenings'] = mornings_evenings.apply(lambda x: 'red' if x['evenings'] < 0 else 'green', axis=1)

mornings_evenings = mornings_evenings[['id', 'c_mornings', 'c_evenings']]

mornings_evenings = mornings_evenings.merge(right=all_locs, how='inner', on='id')

mornings_evenings.head()

In [ ]:
## Set up bokeh plot

df_diff = mornings_evenings.loc[:,('name','lat', 'lon', 'c_mornings', 'c_evenings')]
df_diff['size'] = 5

x_range = (df_diff.lon.min() - 0.001, df_diff.lon.max() + 0.003)
y_range = (df_diff.lat.min() - 0.003, df_diff.lat.max() + 0.003)

points_source = ColumnDataSource(ColumnDataSource.from_df(df_diff))

print(x_range)
print(y_range)

plot_h = 600
plot_w = 900

df_diff.head()

In [ ]:
## Bokeh plot - using panels to flip between mornings and evenings

## Get GlyphRenderer errors again, but can ignore those as it still works fine

from bokeh.models.widgets import Panel, Tabs

tools1 = [PanTool(), WheelZoomTool(), ResetTool()]

p1 = bp.figure(tools=tools1, plot_width=plot_w, plot_height=plot_h,
    x_range=x_range, y_range=y_range, outline_line_color=None,
    min_border=0, min_border_left=0, min_border_right=0,
    min_border_top=0, min_border_bottom=0) 

p1.patches(xs='xs', ys='ys', fill_alpha=0, 
              line_color = '#b3b3b3', line_alpha=0.3, source=json_roads)

circles1 = p1.circle(x='lon', y='lat', size='size', color='c_mornings', alpha=1, source=points_source)

hover1 = HoverTool(
        tooltips=[
            ("Name", "@name")
        ]
    )

p1.add_tools(hover1)
p1.background_fill_color = '#000000'
p1.xaxis.visible = False
p1.yaxis.visible = False
p1.xgrid.grid_line_color = None
p1.ygrid.grid_line_color = None
hover1.renderers.append(circles1)

tab1 = Panel(child=p1, title="Mornings")


tools2 = [PanTool(), WheelZoomTool(), ResetTool()]

p2 = bp.figure(tools=tools2, plot_width=plot_w, plot_height=plot_h,
    x_range=x_range, y_range=y_range, outline_line_color=None,
    min_border=0, min_border_left=0, min_border_right=0,
    min_border_top=0, min_border_bottom=0) 

p2.patches(xs='xs', ys='ys', fill_alpha=0, 
              line_color = '#b3b3b3', line_alpha=0.3, source=json_roads)

circles2 = p2.circle(x='lon', y='lat', size='size', color='c_evenings', alpha=1, source=points_source)

hover2 = HoverTool(
        tooltips=[
            ("Name", "@name"),
        ]
    )

p2.add_tools(hover2)

p2.background_fill_color = '#000000'
p2.xaxis.visible = False
p2.yaxis.visible = False
p2.xgrid.grid_line_color = None
p2.ygrid.grid_line_color = None
hover2.renderers.append(circles2)

tab2 = Panel(child=p2, title="Evenings")

tabs = Tabs(tabs=[ tab1, tab2 ])

output_notebook()
show(tabs)

output_file("bokeh_plots/mornings_evenings.html")
save(tabs)


## Tube strike analysis

Looking at one particular tube strike - see if it has an impact on how many journeys taken etc. 

In [ ]:
## Tube strike - 8th July 2015 18:30 until 9th July 2015 21:30

tube_strike = time_filter(all_data, 7, 10)
tube_strike = date_filter(tube_strike, 2015,7,9,2015,7,10)

## First control group - a week before the strike on the same day

tube_strike_control_1 = time_filter(all_data, 7, 10)
tube_strike_control_1 = date_filter(tube_strike_control_1, 2015,7,2,2015,7,3)

## Second control group - a week after the strike on the same day

tube_strike_control_2 = time_filter(all_data, 7, 10)
tube_strike_control_2 = date_filter(tube_strike_control_2, 2015,7,16,2015,7,17)

## Larger control group - month either side of the strike, dropping weekends and strike itself - 43 days total

control = time_filter(all_data, 7, 10)
control = date_filter(control, 2015,6,9,2015,8,9)
control = weekend_filter(control)
control = drop_specific_date(control, 2015,7,9)

## Two month period including tube strike

period = date_filter(all_data, 2015,6,9,2015,8,9)

In [ ]:
## Number of journeys made on tube strike morning

print(tube_strike.shape)

## Number of journeys week before

print(tube_strike_control_1.shape)

## Number of journeys week after

print(tube_strike_control_2.shape)

In [ ]:
## Bar chart of number of journeys per day, including strike day

from bokeh.charts import Bar

period_collapse = period.loc[:, ('Start Date Converted', 'c')].groupby(['Start Date Converted']).sum()
period_collapse.reset_index(inplace=True)
period_collapse.rename(columns={'c': 'Count', 'Start Date Converted': 'Date'}, inplace=True)

period_collapse.head()

tools = [PanTool(), WheelZoomTool(), ResetTool()]

bar = Bar(period_collapse, values='Count', label='Date', color='red', ylabel='Count',
          legend = None, title="Number of journeys per day", plot_width=950, tools=tools)

output_notebook()
show(bar)

output_file("bokeh_plots/journeys_per_day.html")
save(bar)

In [ ]:
## Statistical test to see if number of journeys on strike day is significantly different from control group average

from math import sqrt

control_collapse = control.loc[:, ('Start Date Converted', 'c')].groupby(['Start Date Converted']).sum()
control_collapse.reset_index(inplace=True)
control_collapse.rename(columns={'c': 'Count', 'Start Date Converted': 'Date'}, inplace=True)

print(control_collapse['Count'].mean())
print(control_collapse['Count'].std())

print(tube_strike['c'].sum())

x_bar = control_collapse['Count'].mean()
mu_0 = tube_strike['c'].sum()
s = control_collapse['Count'].std()
n = 43

t_stat = (x_bar - mu_0) / (s / sqrt(n))

print(t_stat)

In [ ]:
## Show difference in average number of flows in control vs number on tube strike - for top 10 routes 

tube_strike_flows = inflow_outflow(tube_strike)

tube_strike_flows['Flows'] = (tube_strike_flows['inflows'] + tube_strike_flows['outflows'])

tube_strike_flows = tube_strike_flows[['id', 'Flows']]

control_flows = inflow_outflow(control)

control_flows['Flows'] = (control_flows['inflows'] + control_flows['outflows']) / 43

control_flows = control_flows[['id', 'Flows']]

control_flows.sort_values('Flows', axis=0, inplace=True, ascending=False)

control_flows = control_flows[:10]
control_flows = control_flows.merge(right=all_data_names, how='left', on='id')

tube_strike_flows = tube_strike_flows.merge(right=control_flows, how='right', on = 'id')
tube_strike_flows.rename(columns={'Flows_x': 'Flows'}, inplace=True)
del tube_strike_flows['Flows_y']

tube_strike_flows['Type'] = "Tube Strike"
control_flows['Type'] = "Control"

tube_strike_flows

flows = control_flows.append(tube_strike_flows)

In [ ]:
## Bar chart of number of flows for top routes

tools = [PanTool(), WheelZoomTool(), ResetTool()]

bar = Bar(flows, values='Flows', label='Name', group='Type', color=['green', 'red'], ylabel='Flows',
          legend = 'top_right', title="Flows", plot_width=950, tools=tools)

output_notebook()
show(bar)

output_file("bokeh_plots/top_stations.html")
save(bar)

In [ ]:
tube_strike_hist = tube_strike[['Duration']]
tube_strike_hist = tube_strike_hist[tube_strike_hist['Duration'] < 6000]
tube_strike_hist['Type'] = "Tube Strike"

control_hist = control[['Duration']]
control_hist = control_hist[control_hist['Duration'] < 6000]
control_hist['Type'] = "Control"

hist = tube_strike_hist.append(control_hist)

In [ ]:
## Look at distribution of duration

from bokeh.charts import Histogram
from bokeh.layouts import gridplot

hist1 = Histogram(control_hist, values='Duration', ylabel="Flows (Control)", xlabel="",
                  bins=100, width=800, height=300, color="green")
hist2 = Histogram(tube_strike_hist, values='Duration', ylabel="Flows (Tube Strike)", xlabel="Duration (seconds)",
                  bins=100, width=800, height=300, color="red")


p = gridplot([[hist1], [hist2]], toolbar_location="above")

output_notebook()
show(p)

output_file("bokeh_plots/histogram.html")
save(p)

In [ ]:
## Formal two-sample Kolmogorov-Smirnov test

from scipy import stats

control_array = control[['Duration']].values[:, 0]
tube_strike_array = tube_strike[['Duration']].values[:, 0]

stats.ks_2samp(control_array, tube_strike_array)